In [ ]:
import cv2 as cv
import numpy as np
from decimal import Decimal
import os
from matplotlib import pyplot as plt

In [ ]:
path = 'D:\AI_CW\Plant_Images'
files=os.listdir(path)

In [41]:
variable=1
successful=0
unsuccessful=0
asp_ratio_min = 1.99
asp_ratio_max =1.20

In [ ]:
for filename in files:
    if filename.endswith('.jpg'):

        image_name= os.path.join(path,filename)
        #print(image_name)
        image= cv.imread(image_name)
        
        #Converting BGR to HSV values so that only GREEN color is segmented from the image
        image_hsv= cv.cvtColor(image, cv.COLOR_BGR2HSV)
        
        #Defining range of green values
        
        #Lower Mask(50-70)
        lower_green=np.array([50,100,100])
        upper_green=np.array([70,255,255])
        mask0=cv.inRange(image_hsv,lower_green,upper_green)
        
        #Upper Mask(170-180)
        lower_green=np.array([170,100,100])
        upper_green=np.array([180,255,255])
        mask1=cv.inRange(image_hsv,lower_green,upper_green)
        
        #Joining two masks
        mask= mask0 + mask1
        
        #Output After Masking
        output_hsv = image_hsv.copy()
        output_hsv[np.where(mask==50)] = 50
        
        #Converting image to grayscale
        gray= cv.cvtColor(output_hsv,cv.COLOR_BGR2GRAY)
        plt.imshow(gray)
        
        #Histogram Equalization
        equal_histogram = cv.equalizeHist(gray)
        
        #Applying Gaussian Blur with 5*5 Filter
        blur = cv.GaussianBlur(equal_histogram,(5,5),0)
        
        # Conversion to Binary (Thresholding using Ostu's Binarization)
        ret, threshold = cv.threshold(gray, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        
        plt.imshow(threshold)
        
    
        #Finding the total Contours in the image
        contours, _ = cv.findContours(threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        print("No. of contours detected -> %d " %len(contours))
        
        
        #Sorting contours as per area, Only the biggest1 is selected for further processing
        contours= sorted(contours, key = cv.contourArea, reverse = True)[:1]
        
        screen_contours = None
        for c in contours:

            peri=cv.arcLength(c, True)
            approx= cv.approxPolyDP(c, 0.07 *peri, True)

            x, y, w, h = cv.boundingRect(approx)
            a_ratio = float(w) / h
            aspect_ratio_string = str(a_ratio)
            aspect_ratio= Decimal(aspect_ratio_string)

            if (1.2 < aspect_ratio < 1.99):

                if len(approx) == 4:

                    rect = cv.minAreaRect(c)
                    box = cv.boxPoints(rect)
                    box = np.int0(box)

                    print("asp ratio ->", aspect_ratio)

                    screen_contours = approx

                    if aspect_ratio<asp_ratio_min:
                        asp_ratio_min=aspect_ratio

                    if aspect_ratio>asp_ratio_max:
                        asp_ratio_max=aspect_ratio

                    break

                    
        #If Leaf is found, draw Contours on image, else display Error Message
        if(screen_contours is not None):

          
            image_masking = np.zeros(gray.shape, np.uint8)
            new_image = cv.drawContours(image_masking, [box], 0, 255, -1, )
            new_image = cv.bitwise_and(image,image, mask=image_masking)


            
            x, y, w, h = cv.boundingRect(box)
            roi = new_image[y:y + h, x:x + w]

            #Deskewing the Leaf
            # Deskewing the image
            roi_gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
            coords = np.column_stack(np.where(roi_gray > 0))
            angle = cv.minAreaRect(coords)[-1]

            if angle < -45:
                angle = -(90 + angle)
            else:
                angle = -angle
                

            # rotate the image to deskew it
            (h, w) = roi.shape[:2]
            center = (w // 2, h // 2)
            M = cv.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv.warpAffine(roi, M, (w, h),
                                     flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)


            #Saving the leaf

            file_name= 'D:\AI_CW\demo\leaves\leaf_'+str(variable)+".jpg"
            print(filename)
            print(file_name)
            variable+=1
            cv.imwrite(file_name, rotated)
            successful+=1

        else:
            print(filename)
            print("Leaf Not Found!!")
            print("unsuccessful ->", image_name)
            unsuccessful+=1

print('No of successful ->', successful)
print('No of unsuccessful ->', unsuccessful)
print('Success Ratio ->', (successful/(successful+unsuccessful))*100)
print("Aspect Ratio min ->", asp_ratio_min)
print("Aspect Ratio max ->", asp_ratio_max)


       

            

No. of contours detected -> 76 
asp ratio -> 1.5
0001_0001.jpg
D:\AI_CW\demo\leaves\leaf_63.jpg
No. of contours detected -> 90 
asp ratio -> 1.5
0001_0002.jpg
D:\AI_CW\demo\leaves\leaf_64.jpg
No. of contours detected -> 95 
asp ratio -> 1.5
0001_0003.jpg
D:\AI_CW\demo\leaves\leaf_65.jpg
No. of contours detected -> 65 
asp ratio -> 1.5
0001_0004.jpg
D:\AI_CW\demo\leaves\leaf_66.jpg
No. of contours detected -> 65 
asp ratio -> 1.5
0001_0005.jpg
D:\AI_CW\demo\leaves\leaf_67.jpg
No. of contours detected -> 253 
0001_0006.jpg
Leaf Not Found!!
unsuccessful -> D:\AI_CW\Plant_Images\0001_0006.jpg
No. of contours detected -> 102 
asp ratio -> 1.5
0001_0007.jpg
D:\AI_CW\demo\leaves\leaf_68.jpg
No. of contours detected -> 88 
asp ratio -> 1.5
0001_0008.jpg
D:\AI_CW\demo\leaves\leaf_69.jpg
No. of contours detected -> 64 
asp ratio -> 1.5
0001_0009.jpg
D:\AI_CW\demo\leaves\leaf_70.jpg
No. of contours detected -> 74 
asp ratio -> 1.5
0001_0010.jpg
D:\AI_CW\demo\leaves\leaf_71.jpg
No. of contours det

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
